In [ ]:
import numpy as np

import matplotlib.pyplot as plt


In [ ]:
import optim_tools as tools

In [ ]:
# state-based simulation equation:

A = np.matrix([[  2.84217094e-14,   1.71379179e+01],
               [ -1.00000000e+02,  -1.85369064e+02]])

B = np.matrix([[ 17.34396868],
               [  9.87641374]])

C = np.matrix([[ 0., -1.]])

D = np.matrix([[ 0.]])


# initial X
x0 = np.matrix(np.zeros(A.shape[0])).T

#T: time 
T = np.arange(0, 0.6, 0.001) 

#s: input, e.g., step function with amplitude of 0.2
s = 0.1*np.ones(len(T));

In [ ]:
# opened loop 
y, u, u_sat = tools.simulate(A, B, C, D, tools.openLoop, s, T, x0=x0)
plt.figure()
line1, = plt.plot(T, np.array(y[0,:].T), 'b', label='cartesian velocity')
first_legend = plt.legend(handles=[line1], loc=1)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('T')
plt.ylabel('y')
plt.title('Step Responce for Open Loop')
plt.show()

In [ ]:
import glob #for returning files having the specified path extension
import os #checking for empty file
import pandas as pd

task_null           = sorted(glob.glob('../../students/sysid2/step_log_new/*/*task*.log')) #corresponds to .log files that has data related to the first position
control_null        = sorted(glob.glob('../../students/sysid2/step_log_new/*/*control*.log'))
task_remaining      = sorted(glob.glob('../../students/sysid2/step_log_new/*/*task*.log.*')) #corresponds to remaining log.'n' files
control_remaining   = sorted(glob.glob('../../students/sysid2/step_log_new/*/*control*.log.*'))
task_v              = sorted(task_null + task_remaining) #set of all task_velocity logs
control_o           = sorted(control_null + control_remaining) #set of all control logs
observations        = len(task_null) #total number of experiments conducted/observations taken
positions           = int(len(task_v) / observations) #number of points in the given task space
task_full           = [] #A task_velocity list whose each element is a list of similar log files i.e from the same position
control_full        = [] #A control_output list whose each element is a list of similar log files i.e from the same position

for i in range(0, positions):
    task_full.append([])
    control_full.append([])
    for j in range(0, observations):
        task_full[i].append(task_v[i + (j * positions)])
        control_full[i].append(control_o[i + (j * positions)])

count = 0 #counter that returns the number of empty files
for i in range(0, positions):
    for j in range(0, observations):
        if os.stat(task_full[i][j]).st_size == 0:
            count = count + 1
            
for i in range(0, positions):
    for j in range(0, observations-count):
        if os.stat(task_full[i][j]).st_size == 0:
            del(task_full[i][j])
            del(control_full[i][j])
            
# Reading all the data into a dataframe array
df_ist_soll = []
for i in range(0, positions):
    df_ist_soll.append([])
    for j in range(0, observations):
        try:
            df_soll = pd.read_csv(control_full[i][j])
            df_soll.rename(columns=lambda x: x.strip(), inplace=True)

            #df_soll = df_soll.set_index('time')
            
            df_ist = pd.read_csv(task_full[i][j])
            df_ist.rename(columns=lambda x: x.strip(), inplace=True)

            #df_ist = df_ist.set_index('time')
            
            df_ist_soll[i] = pd.concat([df_soll['values[0]'], df_ist['values[2]']], axis=1).fillna(method='pad')
            #df_ist_soll[i].index = pd.to_datetime(df_step.index)
            #df_ist_soll[i].append(ob.batch_read_data(control_full[i][j], task_full[i][j]))
        except:
            continue



In [ ]:
# Testsystem aufsetzen
import control as con

# pt1 System
k = 1
t = 0.8e-2

delay = 0.15

# Polynom
tf_pt1 = con.matlab.tf(k, [t, 1])
#print tf_pt1


# pt2 System
K = 1
d = 1
T = .04
delay2 = 0.055

a0 = 1
a1 = (2 * d * T) #16
a2 = (T**2) #100
b0 = K

# Polynom
tf_pt2 = con.matlab.tf(K, [a2, a1, a0])
#print tf_pt2

# Zustandsraum
ss_1a = con.matlab.tf2ss(tf_pt2)
#print ss_1a

# Füge Zeitversatz zu
d_num2, d_den2 = con.pade(delay2, 1)
tf_delay2 = con.tf(d_num2, d_den2)
ss_pt2_delay = con.series(tf_delay2, tf_pt2)

d_num, d_den = con.pade(delay, 1)
tf_delay = con.tf(d_num, d_den)
tf_pt1_delay = con.tf(d_num, d_den)
ss_pt1_delay = con.series(tf_delay, tf_pt1)

print con.matlab.tf2ss(ss_pt2_delay)


#print con.matlab.tf2ss(ss_delay)

#import matplotlib.pyplot as plt
#%pylab inline
d_yout2, d_T2 = con.matlab.step(ss_pt2_delay)
#yout, T = con.matlab.step(tf_pt2) # step without delay

#plt.plot(d_T, d_yout, 'r-', label='poly_est')
#plt.plot(np.add(d_T, delay), yout, 'r-', label='pt2') #delay in timeaxis!

d_yout, d_T = con.matlab.step(ss_pt1_delay)
#yout, T = con.matlab.step(tf_pt1) # step without delay

#plt.plot(d_T, d_yout, 'r-', label='poly_est')
#plt.plot(np.add(d_T, delay), yout, 'y-', label='pt1') #delay in timeaxis!


In [ ]:
ax = df_ist_soll[6].plot()
pd.DataFrame(np.array(0.1*d_yout2), d_T2*100).plot(ax=ax)
pd.DataFrame(np.array(0.1*d_yout), d_T*100).plot(ax=ax)
#pd.DataFrame(np.array(y[0,:].T), T*100).plot()